In [1]:
cd /home/ubuntu/DistMEC/

/home/ubuntu/DistMEC


In [2]:
import numpy as np
import matplotlib.pyplot as plt

from classes.User import *
from classes.solver import *
from classes.Server import *
from classes.distributed_utils import *

import pickle
import os

In [3]:
def load_trial(path_user, path_regret,  trial_num):
    
    user_load_path = path_user + str(trial_num) + '.p'
    with open(user_load_path, 'rb') as handle:
        user_dict = pickle.load(handle)
    
    regret_load_path = path_regret + str(trial_num) + '.p'
    with open(regret_load_path, 'rb') as handle:
        regret_dict = pickle.load(handle)
    
    return user_dict, regret_dict


def track_collision(users):
    
    collision = np.zeros([len(users),len(users)])
    time_steps = len(users[0].history_location)
    for t in range(time_steps):
        arm_pulls = []
        for u in users:
            arm_pulls += [u.history_pull[t]]
        for u in range(len(users)):
            for u2 in range(len(users)):
                if u != u2 and arm_pulls[u] == arm_pulls[u2]:
                    collision[u,u2] += 1
    return collision

def track_movement_overlap(users):
    mvmt = np.zeros([len(users),len(users)])
    time_steps = len(users[0].history_location)
    for t in range(time_steps):
        locs = []
        for u in users:
            locs += [u.history_location[t]]
        for u in range(len(users)):
            for u2 in range(len(users)):
                if u != u2 and locs[u] == locs[u2]:
                    mvmt[u,u2] += 1
    return mvmt

def track_movement(users, num_servers):
    locs = np.zeros([len(users),num_servers])
    time_steps = len(users[0].history_location)
    for t in range(time_steps):
        for u in range(len(users)):
            then_loc = int(users[u].history_location[t])
            locs[u, then_loc] += 1

    return locs


def track_mu_est(users, num_servers):
    mu_est = np.zeros([len(users),num_servers])
    for u in range(len(users)):
        mu_est[u,:] = users[u].mu
    return mu_est

def pull_records(users, num_servers):
    pull_rec = np.zeros([len(users),num_servers])
    for u in range(len(users)):
        pull_rec[u,:] = users[u].pulls
    return pull_rec

In [4]:
# path_regret = 'results/22_06_13_reserve_troubleshoot/regret_trial'
# path_user = 'results/22_06_13_reserve_troubleshoot/user_trial'

# path_regret = 'results/22_06_14_reserve_troubleshoot_n3_kick/regret_trial'
# path_user = 'results/22_06_14_reserve_troubleshoot_n3_kick/user_trial'


path_regret = 'results/fixed_bug_t2/regret_trial'
path_user = 'results/fixed_bug_t2/user_trial'




In [7]:
mode = 'new'
idxs = 0
a,b = load_trial(path_user, path_regret, idxs)
coll = track_collision(a[mode])
mvmt_overlap = track_movement_overlap(a[mode])
mu_est = track_mu_est(a[mode], 16)
pull_rec = pull_records(a[mode],16)
mvmt = track_movement(a[mode],16)

# print(coll.astype(int))
# print(mu_est)
# print(mu_est.argmax(axis=1))
# print(pull_rec.astype(int))
# print(pull_rec.argmax(axis=1))
# print(mvmt/5000)
# print(mvmt.argmax(axis=1))


[[  13   12   40    8  114   47    8  752  743   14  160    9  187 2412
   291  189]
 [ 127  357   84   30    7  315   30  426   16   80    9   41 3154   45
    16  262]
 [ 332  563  222   59   27  182   73  361   87   15   31  277  144  168
   137 2321]
 [ 197  185   46   31   24   64   36 2807   93  199  310  157   15  348
    69  418]
 [  12  792    8    6   12  868   52   17  567  172   67   24   12   20
  2163  207]
 [ 106   25    8   83   20  199    7   25   40   16 3915    7   39  405
     9   95]
 [  12 1926    9   24   15  798   16   59  233   84   70   66  291   23
  1362   11]
 [  87  209   87   10    8  746   38  176 2960    8   60    8  269   88
    57  188]
 [  21  847   12   14  209  700   11  192   23 1070  286   11  198 1352
    16   37]
 [  16   96   11   14   29 1101   29  289  223   11  105   26  706  157
   890 1296]]


In [6]:
print(mvmt)

[[  35.  698.  107.  327.  141.  341.   58.  789.   27.  118.  574.  238.
   943.  515.   42.   46.]
 [  20.  292.  135.  334.  426.   41.  200.  968.  183.  141.  101.  972.
   181.  230.   24.  751.]
 [1087.  148.  520.  123.  483.   59.  435.  228.  473.   80.  231.  275.
   141.  299.  360.   57.]
 [  42.  272.  155.  291.  423.  234.  529. 1092.   67.  879.  123.  186.
   240.   73.   36.  357.]
 [  34.  231.  104.   40.   20.  145.  635.  235.   79.  180.  588.  468.
   749.   58.  309. 1124.]
 [1308.  153.  119.  654.   44.   47.   66.   40.  313.  745.  124.  101.
    46.  731.  214.  294.]
 [  76.  228.   60.  373.  789.  127.  549.   32.  748.  152.  118.  379.
   191.  890.  170.  117.]
 [ 214.  143.   61.   50.  256.   50.  600.  229.   80.   60.  313.   70.
   198. 1325.  964.  386.]
 [ 176.   87.   98.  311. 1153.  414.  424.   44.  524.  351.  498.   30.
   297.  330.   47.  215.]
 [  79.  214.  174.  238.  124.  253. 1153.   10.  199.  576.  522.  375.
   159.  833.   2

In [64]:
np.sum(a[mode][-3].history_reserve[:])/5000

0.9014

In [60]:
print(a[mode][0].mu_est)
print(a[mode][0].mu)

[0.71900826 0.63414634 0.67391304 0.83333333 0.27777778 0.13513514
 0.02222222 0.         0.36842105 0.24242424 0.99680511 0.97215496
 0.08333333 0.37183099 0.55924171 0.55865922]
[0.71207203 0.46495019 0.68835921 0.8229379  0.19677066 0.15974612
 0.01622407 0.16870992 0.38911061 0.23768286 0.99510612 0.97321672
 0.07871893 0.36628599 0.52465089 0.57533903]


In [55]:
print(a[mode][0].expected_time)
print(a[mode][0].svr_stick_idx)

2
11


In [56]:
a[mode][0].update_waittime(0,0,0)

In [57]:
a[mode][0].ucb_present

array([0.9066094 , 0.95642847, 0.97817611, 0.90479039, 0.96564861,
       0.6149153 , 0.45726994, 1.03180625, 1.03794536, 0.60165307,
       1.1617622 , 1.04395723, 0.92579961, 0.52672298, 0.76015188,
       0.77678984])